<a href="https://colab.research.google.com/github/satyamnewale/numpy-Book/blob/main/day_3-16Nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

day_3-16_Nov

non-contiguous -> copy

contiguous -> view

In [9]:
import numpy as np

a1 = np.arange(0,20,2)

#slicing
b= a1[2:7]
print(f"array a1 before: {a1}, array b before: {b}")
#modify
b[0]= 9
print(f"array a1 after: {a1}, array b after: {b}")

#check whether slice create view or copy
truth_slice = np.shares_memory(a1,b)
print(f"slice create view? {truth_slice}, if true memory is shared and its view or vice-versa copy is created with different memory.")

print("-------------------------------------------------------------------")

a2 = np.arange(0,20,2)
#indexing
c = a2[[1,3,5,6,7,8,9]]
print(f"array a2 before: {a2}, array c before: {c}")
#indexing
c[0] = 9
print(f"array a2 after: {a2}, array c after: {c}")

#check whether indexing create view or copy
truth_index = np.shares_memory(a2,b)
print(f"slice create view? {truth_index}, if true memory is shared and its view or vice-versa copy is created with different memory.")

print("-------------------------------------------------------------------")
a3 = np.arange(0,20,2)
#boolean masking
mask = a3%4 == 0
d = a3[mask]
print(f"array a3 before: {a3}, array d before: {d}")

#modify
d[0] = 7
print(f"array a3 after: {a3}, array d after: {d}")

#check
truth_mask = np.shares_memory(a3,d)
print(f"slice create view? {truth_mask}, if true memory is shared and its view or vice-versa copy is created with different memory.")


array a1 before: [ 0  2  4  6  8 10 12 14 16 18], array b before: [ 4  6  8 10 12]
array a1 after: [ 0  2  9  6  8 10 12 14 16 18], array b after: [ 9  6  8 10 12]
slice create view? True, if true memory is shared and its view or vice-versa copy is created with different memory.
-------------------------------------------------------------------
array a2 before: [ 0  2  4  6  8 10 12 14 16 18], array c before: [ 2  6 10 12 14 16 18]
array a2 after: [ 0  2  4  6  8 10 12 14 16 18], array c after: [ 9  6 10 12 14 16 18]
slice create view? False, if true memory is shared and its view or vice-versa copy is created with different memory.
-------------------------------------------------------------------
array a3 before: [ 0  2  4  6  8 10 12 14 16 18], array d before: [ 0  4  8 12 16]
array a3 after: [ 0  2  4  6  8 10 12 14 16 18], array d after: [ 7  4  8 12 16]
slice create view? False, if true memory is shared and its view or vice-versa copy is created with different memory.


refinement:
---
1. slicing create a view :
*   That means both arrays point to the same memory.
*   Modifying the slice changes the original array because they share memory.
*   warning - when editing sliced array, changes in original array will occure.
*   note - if change value in view using index eg: b[0] = 9 ,but the index in original array maybe not same.
*   slice with step = 1 is contiguous eg: a[1:3] ,step != 1 is not contiguous eg: arr[:, ::3].

2. fancy indexing creates copy:
*   NumPy allocates a new array for the requested elements.
*   The new array does not share memory with the original.
*   this create copy because it’s not contiguous as a view, but the result copy IS contiguous.

3. masking creates copy ,in few situation it creates view (np.lib.stride_tricks):
*   NumPy allocates a new array for the requested elements.
*   The new array does not share memory with the original.
*   Output is contiguous, but original memory not shared

---
np.shares_memory()

Purpose:
Confirms if two arrays share the same memory block.

Returns True or False.



In [18]:
import numpy as np

arr = np.arange(12).reshape(3,4)
print(f"arra C_continuous: {arr}")
print(f"if my calculations are correct going down a row shall require 32 bits as travelling 4 elements means 4x8 = 32 bits and to travel side p column it will require 8 byte as 1x8 = 8 bytes, ans:{arr.strides}")
print(f"lets check: {arr.dtype.itemsize}, so it is int64 , my answer was correct.")

print(f"flags : {arr.flags}, memory is stored in C_CONTIGUOUs")


arr_f = np.asfortranarray(arr)
print(f"arr_f: {arr_f}, strides of fortran array: {arr_f.strides}")
print(f"flag of fortran array: {arr_f.flags}")


arra C_continuous: [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
if my calculations are correct going down a row shall require 32 bits as travelling 4 elements means 4x8 = 32 bits and to travel side p column it will require 8 byte as 1x8 = 8 bytes, ans:(32, 8)
lets check: 8, so it is int64 , my answer was correct.
flags :   C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
, memory is stored in C_CONTIGUOUs
arr_f: [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]], strides of fortran array: (8, 24)
flag of fortran array:   C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



in strides:
numpy stores memory in contiguous way ,either row-major(c) or column-major(fortran).
strides gives tuple , one number per axis.


.....

1. C_CONTIGUOUS :Memory is stored in row-major order (C-order) ,Entire first row is stored first in RAM,Then the second row, no gaps, no jumps.
*   Fastest for operations expecting C-order (np.dot, ML libraries)
*   Fastest for slicing rows

2. Array is stored in column-major order (Fortran), Columns are NOT contiguous in memory. To move down a column, NumPy must jump around in memory

3. OWNDATA: tels whether array has own data or not.
false means : array is slice, transpose, views producd by reshape.

4. WRITEABLE: can modify the data or not.

---
C-order (default):

Memory stored row by row. First row, then second row, etc.

Strides: strides[0] > strides[1] → moving down requires more bytes than moving right.

Fortran-order:
Memory stored column by column. First column, then second column, etc.

Strides: strides[0] < strides[1] → moving down is cheaper than moving right.

---

reality: ML frameworks like PyTorch and TensorFlow assume row-major, but some linear algebra ops (like LAPACK) expect Fortran-order. Strides can make huge speed differences.

even the array is changed into fortran continuous the shape of array dont changes.

In [33]:
import numpy as np

arr = np.arange(12)
a = arr.reshape(3,4)
print(np.shares_memory(arr,a))

transpose = a.T
print(np.shares_memory(arr,transpose))
print(f"strides of a: {a.strides}, strides of a.T :{(a.T).strides}")
print(f"a: {a}, a.T: {transpose}, first 3 values will be 0,4,8 ")
print(f"flag of a: {a.flags}, flag of a.T :{(a.T).flags}")

#slice and strides
arr_slice = a[:,::2]
print(f"array a: {a}, sliced array: {arr_slice}")
print(f"strides of a: {a.strides}, strides of arr_slice: {arr_slice.strides}, # strides scaled by slice step,and not continuous.")

True
True
strides of a: (32, 8), strides of a.T :(8, 32)
a: [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]], a.T: [[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]], first 3 values will be 0,1,2 
flag of a:   C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
, flag of a.T :  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

array a: [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]], sliced array: [[ 0  2]
 [ 4  6]
 [ 8 10]]
strides of a: (32, 8), strides of arr_slice: (32, 16), # strides scaled by slice step


array reshape creates a view not copy.

transpose also creates a view .

transpose of a array convert C_CONTINUOUS to FORTRAN_CONTINUOUS, now operation on arrays will be slow and it is continuous but fortran-continuous.

b[i,j] and strides[32,8], itemsize is 8 :-

offset=i×32+j×8   , i = rows and j = columns

In [8]:
import numpy as np

# Contiguous → reshape gives a VIEW
a = np.arange(24)
b = a.reshape(2,3,4)

print(f"b is view of a : {b.base is a}")  # True → view
print(f"does b has its own data: {b.flags['OWNDATA']}")  # False

# Non-contiguous slice → reshape gives a COPY
a1 = np.arange(24)[::2]  # step makes it non-contiguous
b1 = a1.reshape(2, 2, 3)

print(f"b is view of a1 : {b1.base is a1}")  # False → copy
print(f"does b1 has its own data: {b1.flags['OWNDATA']}")  # True

a2 = np.arange(12)
b2 = a2.ravel()
print(b2)
b2[0] = 9
print(f"b2 edit: {b2}")
print(b2.base is a2)  # True → view

# Non-contiguous → ravel gives COPY
a3 = np.arange(12)[::2]
b3 = a3.ravel()
print(b3)
b3[0] = 8
print(f"b3 edit: {b3}")
print(b3.base is a3)  # False → copy

a4 = np.arange(12)
b4 = a.flatten()
print(f"b4:{b4}")
b4[1] = 9
print(f"b4 after:{b4}")
print(b.base is a)  # False → always copy
print(b.flags['OWNDATA'])  # True


a5 = np.arange(12).reshape(3, 4)

# (3,4) → (3,4,1)
b5 = a5[:, :, np.newaxis]

# (3,4) → (1,3,4)
c5 = a5[np.newaxis, :, :]

print(a5.shape, b5.shape, c5.shape)


a6 = np.arange(12).reshape(1, 3, 4, 1)

b6 = a6.squeeze()

print(a6.shape)  # (1,3,4,1)
print(b6.shape)  # (3,4)

# TASK
x = np.arange(24)
y = x.reshape(2, 3, 4)

print(y.flags['OWNDATA'])  # False → view
print(y.shape)
z = y.transpose(2,0,1)

print(z.shape)
print(z.base is x)  # True → view


b is view of a : True
does b has its own data: False
b is view of a1 : False
does b1 has its own data: False
[ 0  1  2  3  4  5  6  7  8  9 10 11]
b2 edit: [ 9  1  2  3  4  5  6  7  8  9 10 11]
True
[ 0  2  4  6  8 10]
b3 edit: [ 8  2  4  6  8 10]
False
b4:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
b4 after:[ 0  9  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
True
False
(3, 4) (3, 4, 1) (1, 3, 4)
(1, 3, 4, 1)
(3, 4)
False
(2, 3, 4)
(4, 2, 3)
True


why always flatten gives copy?

NumPy intentionally designed flatten() as a “safe” operation.
It guarantees that the returned array is completely independent from the original.

In [15]:
#mini edge case tests

import numpy as np

a = np.arange(12)
b = a[-5:-1] # output will be [7,8,9,10]
print(f"a: {a}, b: {b}")
print(b.base is a) # true means it a view of a

#empty slice
c = a[5:5]
print(f"value of c:{c}, shape of c:{c.shape}, type: {c.base is a} -  true its a view")

#reshape
x = np.arange(6)
x1 = x.reshape(2,3)
x2 = x.reshape(3,2)
x3 = x.reshape(6,1)
print(f"x: {x}, x1: {x1}, x2: {x2}, x3: {x3}")
print(f"type of x1: {x1.base is x}; type of x2: {x2.base is x}; type of x3: {x3.base is x3} false because it is non-contiguous ; - if true its a view and false its copy.")

#4. transpose and flatten
m = np.arange(6)
n = np.transpose(m)
print(f"value of m: {m}, shape of m : {m.shape}, value of n:{n}, shape of n: {n.shape}")
print(f"copy(false) or view(true) n of m:{n.base is m}") # predict true
o = m.flatten()
print(f"value of o: {o}, shape of o: {o.shape}")
print(f"copy(false) or view(true) o of m:{o.base is m}") # predict false as flatten give copy in numpy



a: [ 0  1  2  3  4  5  6  7  8  9 10 11], b: [ 7  8  9 10]
True
value of c:[], shape of c:(0,), type: True -  true its a view
x: [0 1 2 3 4 5], x1: [[0 1 2]
 [3 4 5]], x2: [[0 1]
 [2 3]
 [4 5]], x3: [[0]
 [1]
 [2]
 [3]
 [4]
 [5]]
type of x1: True; type of x2: True; type of x3: False false because it is non-contiguous ; - if true its a view and false its copy.
value of m: [0 1 2 3 4 5], shape of m : (6,), value of n:[0 1 2 3 4 5], shape of n: (6,)
copy(false) or view(true) n of m:True
value of o: [0 1 2 3 4 5], shape of o: (6,)
copy(false) or view(true) o of m:False


* Transpose returns a VIEW (almost always) because transpose does NOT move data in memory — it only changes the strides.

* Transposing a non-contiguous slice that can't be represented with a simple stride permutation gives copy.

* or Transposing a non-contiguous slice that can't be represented with a simple stride permutation.

transpose are not moving any data.
they are reinterpreting the same memory by changing how to jump (stride) through elements.

---
5 Revision Questions:

1. Slice vs fancy indexing: which creates a view?
* slice with step = 1 is contiguous it create view eg: a[1:3] ,step != 1 is not contiguous it creates copy eg: arr[:, ::3].
* fancy indexing create copy.

2. What does .strides tell you?
* strides tell about the byte required to jump from one element to another. it depents on bytes, also offset to move down road is more then moving side of row.

3. Difference between ravel() and flatten()?
* ravel() gives a VIEW if the array is contiguous.
* ravel() gives a COPY if the array is not contiguous.
* flatten() gives copy always, new independent array, which is C-contiguous (row-major).
all elements in a fresh memory block is given.
* ravel() is fast in contiguous memory and flatten() is slow.

4. How does C vs F order affect reshaping and memory?
* C is fast due to continuous memory as its row wise , whereas F is slow due to memory management in column wise manner.

5. Why is knowing view vs copy important in ML preprocessing?
* knowing view and copy is important to undertand the representation of memory, whether its continuous or non- contiguous. This affects the speed of code if heavy iteration is involved.